# BasicTick: Create Cluster: GW
This notebook will create (start) a Gateway cluster.

In [ ]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [ ]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env import *

# Cluster names and database
from basictick_setup import *

# Managed KX Database and Cluster names to create
SEC_THREADS='4'

# Cluster Settings
CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'gwmkdbcluster.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'rdb_name', 'value': RDB_CLUSTER_NAME}, 
    { 'key': 'hdb_name', 'value': HDB_CLUSTER_NAME}, 
]

In [ ]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

# Deploy code

In [ ]:
# zip the code
#os.system(f"zip -r -X {CODEBASE}.zip {CODEBASE} -x '*.ipynb_checkpoints*'")
os.system(f"cd {CODEBASE}; zip -r -X ../{CODEBASE}.zip . -x '*.ipynb_checkpoints*';")

# copy code to S3

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 cp  --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
else:
    cp = f"""
aws s3 cp  --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
    
# execute the S3 copy
os.system(cp)

## Create Cluster

In [ ]:
# be sure the HDB and RDB are running
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME)
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME)

print("*** HDB and RDB are running ***")

In [ ]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=GW_CLUSTER_NAME,
    clusterType='GATEWAY',
    releaseLabel = '1.0',
    capacityConfiguration=CAPACITY_CONFIG,
    clusterDescription="Created with create_GW notebook",
    executionRole=EXECUTION_ROLE,
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [ ]:
resp

In [ ]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME, show_wait=True)
print("** DONE **")

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="GATEWAY")

display(cdf)

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )